### Análise de Times

In [ ]:

import pandas as pd
import panel as pn
import plotly.graph_objects as go
from datetime import datetime

pn.extension('plotly')

# --- Carregar e preparar os dados ---
try:
    game_df = pd.read_csv('csv/game.csv', parse_dates=['game_date'])
    game_df.columns = [col.lower() for col in game_df.columns]
    team_df = pd.read_csv('csv/team.csv')
    team_df.columns = [col.lower() for col in team_df.columns]
except Exception as e:
    raise RuntimeError(f"Erro ao carregar os arquivos CSV: {e}")

# Renomear colunas para facilitar a fusão
team_df.rename(columns={'id': 'team_id', 'full_name': 'team_name'}, inplace=True)

# Separar dados de casa e fora
home_df = game_df[['game_date', 'team_id_home', 'wl_home', 'fgm_home', 'fga_home', 'pts_home', 'ast_home', 'reb_home', 'game_id']].copy()
away_df = game_df[['game_date', 'team_id_away', 'wl_away', 'fgm_away', 'fga_away', 'pts_away', 'ast_away', 'reb_away', 'game_id']].copy()

home_df.rename(columns={
    'team_id_home': 'team_id', 'wl_home': 'wl', 'fgm_home': 'fgm', 'fga_home': 'fga',
    'pts_home': 'pts', 'ast_home': 'ast', 'reb_home': 'reb'
}, inplace=True)
home_df['location'] = 'Casa'

away_df.rename(columns={
    'team_id_away': 'team_id', 'wl_away': 'wl', 'fgm_away': 'fgm', 'fga_away': 'fga',
    'pts_away': 'pts', 'ast_away': 'ast', 'reb_away': 'reb'
}, inplace=True)
away_df['location'] = 'Fora'

# Combinar dados de casa e fora em um único DataFrame
df = pd.concat([home_df, away_df], ignore_index=True)

# Adicionar nomes dos times
df = pd.merge(df, team_df[['team_id', 'team_name']], on='team_id', how='left')

# Extrair o ano da data do jogo
df['year'] = df['game_date'].dt.year
df['game_date_only'] = df['game_date'].dt.date

# --- Widgets Interativos ---
team_names = sorted(df['team_name'].dropna().unique())
team_selector = pn.widgets.MultiSelect(
    name='Selecione os Times', 
    options=team_names, 
    value=[team_names[0], team_names[1], team_names[2]],
    size=10  # Aumenta a altura do widget
)

min_date = df['game_date_only'].min()
max_date = df['game_date_only'].max()
date_slider = pn.widgets.DateRangeSlider(
    name='Intervalo de Data dos Jogos',
    start=min_date, end=max_date,
    value=(min_date, max_date)
)

# --- Funções Reativas para os Gráficos ---
@pn.depends(team_selector.param.value, date_slider.param.value)
def create_fg_pct_chart(teams, date_range):
    filtered_df = df[(df['team_name'].isin(teams)) & (df['game_date_only'] >= date_range[0]) & (df['game_date_only'] <= date_range[1])]
    
    # Agrupar por time e ano
    yearly_stats = filtered_df.groupby(['team_name', 'year']).agg({'fgm': 'sum', 'fga': 'sum'}).reset_index()
    yearly_stats['fg_pct'] = (yearly_stats['fgm'] / yearly_stats['fga'])
    
    fig = go.Figure()
    for team in teams:
        team_stats = yearly_stats[yearly_stats['team_name'] == team]
        fig.add_trace(go.Bar(x=team_stats['year'], y=team_stats['fg_pct'], name=team))
    
    fig.update_layout(
        title='% Arremessos Convertidos por Ano',
        xaxis_title='Ano',
        yaxis_title='% Convertidos',
        template='plotly_dark',
        barmode='group'
    )
    return fig

@pn.depends(team_selector.param.value, date_slider.param.value)
def create_totals_chart(teams, date_range):
    filtered_df = df[(df['team_name'].isin(teams)) & (df['game_date_only'] >= date_range[0]) & (df['game_date_only'] <= date_range[1])]
    
    team_stats = filtered_df.groupby('team_name').agg({
        'pts': 'sum',
        'ast': 'sum',
        'reb': 'sum',
        'wl': lambda x: (x == 'W').sum(),
        'game_id': 'nunique'
    }).reset_index()
    team_stats.rename(columns={'wl': 'wins', 'game_id': 'games'}, inplace=True)

    fig = go.Figure()
    fig.add_trace(go.Bar(name='Total de Pontos', x=team_stats['team_name'], y=team_stats['pts'], text=team_stats['pts'], textposition='auto'))
    fig.add_trace(go.Bar(name='Total de Assistências', x=team_stats['team_name'], y=team_stats['ast'], text=team_stats['ast'], textposition='auto'))
    fig.add_trace(go.Bar(name='Total de Rebotes', x=team_stats['team_name'], y=team_stats['reb'], text=team_stats['reb'], textposition='auto'))
    fig.add_trace(go.Bar(name='Total de Vitórias', x=team_stats['team_name'], y=team_stats['wins'], text=team_stats['wins'], textposition='auto'))
    fig.add_trace(go.Bar(name='Total de Jogos', x=team_stats['team_name'], y=team_stats['games'], text=team_stats['games'], textposition='auto'))

    fig.update_layout(
        barmode='stack',
        title='Total de Pontos, Assistências, Rebotes, Vitórias e Jogos',
        xaxis_title='Time',
        yaxis_title='Total',
        template='plotly_dark',
        legend_title_text='Métricas'
    )
    return fig

@pn.depends(team_selector.param.value, date_slider.param.value)
def create_home_away_chart(teams, date_range):
    filtered_df = df[(df['team_name'].isin(teams)) & (df['game_date_only'] >= date_range[0]) & (df['game_date_only'] <= date_range[1])]
    
    home_away_stats = filtered_df.groupby(['team_name', 'location'])['pts'].sum().unstack(fill_value=0).reset_index()
    
    # Usar abreviações dos times para o eixo Y
    team_abbreviations = pd.merge(home_away_stats, team_df[['team_name', 'abbreviation']], on='team_name')['abbreviation']

    fig = go.Figure()
    fig.add_trace(go.Bar(name='Pontos em Casa', y=team_abbreviations, x=home_away_stats['Casa'], orientation='h'))
    fig.add_trace(go.Bar(name='Pontos Fora', y=team_abbreviations, x=home_away_stats['Fora'], orientation='h'))

    fig.update_layout(
        barmode='group',
        title='Pontos Marcados em Casa vs. Fora',
        yaxis_title='Abreviação',
        xaxis_title='Pontos',
        template='plotly_dark',
        legend_title_text='Local'
    )
    return fig

# --- Layout do Dashboard ---
# sidebar = pn.Column(
#     pn.pane.Markdown("## Filtros"),
#     team_selector,
#     date_slider
# )

team_analysis_dashboard = pn.Column(
    pn.Row(pn.Column(
    pn.pane.Markdown("## Análise de Times"), 
    pn.pane.Markdown("## Filtros"),
    team_selector,
    date_slider
), create_fg_pct_chart),
    pn.Row(
        create_totals_chart,
        create_home_away_chart
    )
)

# Visão Geral

In [1]:
# Importar bibliotecas
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import panel as pn
pn.extension('plotly')

# Ajuste de display opcional (descomente se preferir)
# pd.set_option('display.max_columns', 200)
# pd.set_option('display.width', 200)

In [2]:
# Carregar dados
GAME_CSV = 'csv/game.csv'
LINE_CSV = 'csv/line_score.csv'

game = pd.read_csv(GAME_CSV, parse_dates=['game_date'], low_memory=False)
line = pd.read_csv(LINE_CSV, parse_dates=['game_date_est'], low_memory=False)

# Converter colunas importantes para numérico (forçando coercion onde houver strings vazias)
num_cols = ['pts_home','pts_away','ast_home','ast_away','fgm_home','fga_home','fgm_away','fga_away']
for c in num_cols:
    if c in game.columns:
        game[c] = pd.to_numeric(game[c], errors='coerce')
# Também garantir no line (pts em line_score)
if 'pts_home' in line.columns:
    line['pts_home'] = pd.to_numeric(line['pts_home'], errors='coerce')
if 'pts_away' in line.columns:
    line['pts_away'] = pd.to_numeric(line['pts_away'], errors='coerce')

# 1) Métricas principais
total_games = game['game_id'].nunique()
total_assists = game['ast_home'].sum(skipna=True) + game['ast_away'].sum(skipna=True)
total_points = game['pts_home'].sum(skipna=True) + game['pts_away'].sum(skipna=True)
# Média de pontos por jogo por time (média histórica por time por partida)
avg_points_per_team_game = ((game['pts_home'] + game['pts_away']) / 2.0).mean()
# Percentual de arremessos convertidos (FGM / FGA) somando home + away
fg_made = game['fgm_home'].sum(skipna=True) + game['fgm_away'].sum(skipna=True)
fg_att = game['fga_home'].sum(skipna=True) + game['fga_away'].sum(skipna=True)
fg_pct = fg_made / fg_att if fg_att and not np.isnan(fg_att) else np.nan

# Formatação simples para exibição (PT-BR style: ',' decimal)
def br_num(x, decimals=2):
    if pd.isna(x):
        return '-'
    s = f'{x:,.{decimals}f}'
    # trocar ponto decimal por vírgula e manter separador de milhar como '.'
    s = s.replace(',', 'X').replace('.', ',').replace('X', '.')
    return s

kpis = {
    'Média de Pontos/Jogo (por time)': br_num(avg_points_per_team_game, 2),
    'Total de Assistências': br_num(total_assists, 0),
    'Total de Jogos': br_num(total_games, 0),
    'Percentual Arremessos Convertidos (FG%)': (br_num(fg_pct * 100.0, 2) + '%') if not pd.isna(fg_pct) else '-',
    'Total de Pontos': br_num(total_points, 0),
}

# Mostrar KPIs (a célula pode ser editada para exibir de forma diferente)
# kpis

In [3]:
# 2) Preparar dados por ano (usando line_score que traz pts_home / pts_away e a data)
line_clean = line.copy()
line_clean['year'] = pd.to_datetime(line_clean['game_date_est'], errors='coerce').dt.year
by_year = line_clean.groupby('year', dropna=True).agg({'pts_home':'sum','pts_away':'sum'}).reset_index().sort_values('year')
by_year['total'] = by_year['pts_home'] + by_year['pts_away']
# by_year.head()

In [4]:
# 3) Gráfico: barras empilhadas (pontos casa / pontos fora) + linha do total
fig = go.Figure()
fig.add_trace(go.Bar(name='Pontos em Casa', x=by_year['year'], y=by_year['pts_home'], marker_color='#1f77b4'))
fig.add_trace(go.Bar(name='Pontos Fora', x=by_year['year'], y=by_year['pts_away'], marker_color='#9467bd'))
fig.add_trace(go.Scatter(name='Total Pontos', x=by_year['year'], y=by_year['total'], mode='lines+markers', marker_color='#2ca02c'))
fig.update_layout(barmode='stack', title_text='Total de Pontos por Ano (Casa vs Fora)', xaxis_title='Ano', yaxis_title='Pontos', legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1), template='plotly_dark', height=450)
fig.update_yaxes(tickformat=',')
# fig

In [ ]:
# 4) Montar layout interativo com Panel: filtro por time + KPIs + gráfico
# Preparar lista de times (nomes completos) a partir do dataframe 'game'
teams = sorted(pd.unique(game['team_name_home'].dropna().tolist() + game['team_name_away'].dropna().tolist()))

# Widget: seleção múltipla de times (vazio = todos)
team_sel = pn.widgets.MultiSelect(name='Filtrar por Time (vazio = todos)', options=teams, size=8)

# Para mapear nomes por jogo no line_score, fazemos merge com game (para garantir nomes completos)
# Certificar que line_clean existe (criado na célula anterior)
line_merged = line_clean.merge(game[['game_id','team_name_home','team_name_away']], on='game_id', how='left')

def make_dashboard(selected_teams):
    # Se nada selecionado -> mostrar todos (usar valores pré-calculados)
    if not selected_teams:
        kpis_local = kpis
        df_by_year = by_year.copy()
    else:
        sel = set(selected_teams)
        # Filtrar jogos onde o time aparece como home ou away
        mask_home = game['team_name_home'].isin(sel)
        mask_away = game['team_name_away'].isin(sel)
        games_sel = game[mask_home | mask_away]
        # Métricas por time(s)
        team_games = games_sel['game_id'].nunique()
        team_pts = games_sel.loc[mask_home, 'pts_home'].sum(skipna=True) + games_sel.loc[mask_away, 'pts_away'].sum(skipna=True)
        team_assists = games_sel.loc[mask_home, 'ast_home'].sum(skipna=True) + games_sel.loc[mask_away, 'ast_away'].sum(skipna=True)
        team_fgm = games_sel.loc[mask_home, 'fgm_home'].sum(skipna=True) + games_sel.loc[mask_away, 'fgm_away'].sum(skipna=True)
        team_fga = games_sel.loc[mask_home, 'fga_home'].sum(skipna=True) + games_sel.loc[mask_away, 'fga_away'].sum(skipna=True)
        team_fg_pct = team_fgm / team_fga if team_fga and not np.isnan(team_fga) else np.nan
        team_avg_pts = team_pts / team_games if team_games else np.nan
        kpis_local = {
            'Média de Pontos/Jogo (por time)': br_num(team_avg_pts, 2),
            'Total de Assistências': br_num(team_assists, 0),
            'Total de Jogos': br_num(team_games, 0),
            'Percentual Arremessos Convertidos (FG%)': (br_num(team_fg_pct * 100.0, 2) + '%') if not pd.isna(team_fg_pct) else '-',
            'Total de Pontos': br_num(team_pts, 0),
        }
        # Agrupar por ano: pontos em casa quando time é home, pontos fora quando time é away
        pts_home_by_year = line_merged[line_merged['team_name_home'].isin(sel)].groupby('year')['pts_home'].sum()
        pts_away_by_year = line_merged[line_merged['team_name_away'].isin(sel)].groupby('year')['pts_away'].sum()
        years = sorted(set(pts_home_by_year.index).union(set(pts_away_by_year.index)))
        df_by_year = pd.DataFrame({'year': years}).set_index('year')
        df_by_year['pts_home'] = pts_home_by_year.reindex(years, fill_value=0)
        df_by_year['pts_away'] = pts_away_by_year.reindex(years, fill_value=0)
        df_by_year = df_by_year.reset_index()
        df_by_year['total'] = df_by_year['pts_home'] + df_by_year['pts_away']
    # Criar figura a partir de df_by_year
    fig_local = go.Figure()
    fig_local.add_trace(go.Bar(name='Pontos em Casa', x=df_by_year['year'], y=df_by_year['pts_home'], marker_color='#1f77b4'))
    fig_local.add_trace(go.Bar(name='Pontos Fora', x=df_by_year['year'], y=df_by_year['pts_away'], marker_color='#9467bd'))
    fig_local.add_trace(go.Scatter(name='Total Pontos', x=df_by_year['year'], y=df_by_year['total'], mode='lines+markers', marker_color='#2ca02c'))
    fig_local.update_layout(barmode='stack', title_text='Total de Pontos por Ano (Casa vs Fora)', xaxis_title='Ano', yaxis_title='Pontos', legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1), template='plotly_dark', height=450)
    fig_local.update_yaxes(tickformat=',')
    # Montar cartões KPI
    cards_local = []
    for title, value in kpis_local.items():
        md_html = f"""
        <div style="background:#0f1720;padding:12px;border-radius:8px;color:#dfefff;width:240px;height:140px;display:flex;flex-direction:column;justify-content:center;align-items:center">
          <strong>{title}</strong>
          <h1 style="margin:4px 0 0 0">{value}</h1>
        </div>
        """
        cards_local.append(pn.pane.HTML(md_html, width=240))
    kpi_row_local = pn.Row(pn.Row(team_sel), *cards_local, sizing_mode='stretch_width')
    chart_local = pn.pane.Plotly(fig_local, config={'displayModeBar': False}, sizing_mode='stretch_width')
    return pn.Column(pn.pane.Markdown('## Visão Geral'), kpi_row_local, pn.Spacer(height=10), chart_local)

# Usar pn.bind para criar um painel reativo que atualiza quando o widget muda
dashboard_reactive = pn.bind(make_dashboard, team_sel)

# Layout final com o seletor de times acima
overview_dashboard = pn.Column(dashboard_reactive)


/tmp/ipykernel_74790/4155204761.py:3: FutureWarning:

unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.



# Análise Geográfica

In [ ]:
import pandas as pd
import panel as pn
import plotly.express as px
from datetime import datetime
pn.extension('plotly')
# Carregar dados
team = pd.read_csv('csv/team.csv')
game = pd.read_csv('csv/game.csv', parse_dates=['game_date'])
game_info = pd.read_csv('csv/game_info.csv', parse_dates=['game_date'])
# Garantir colunas minúsculas
team.columns = [c.lower() for c in team.columns]
game.columns = [c.lower() for c in game.columns]
game_info.columns = [c.lower() for c in game_info.columns]
# Carregar coordenadas das cidades (exemplo: pode ser necessário um dataset externo, aqui simulado)
# Supondo que team.csv tem 'city', 'latitude', 'longitude'
if 'latitude' not in team.columns or 'longitude' not in team.columns:
    # Simulação: adicionar coordenadas fictícias para cidades únicas
    import numpy as np
    cities = team['city'].unique()
    coords = pd.DataFrame({'city': cities, 'latitude': np.random.uniform(25, 50, len(cities)), 'longitude': np.random.uniform(-125, -70, len(cities))})
    team = pd.merge(team, coords, on='city', how='left')

In [ ]:
# Filtro de cidade
cities = sorted(team['city'].dropna().unique())
city_selector = pn.widgets.MultiSelect(name='Selecione a Cidade', options=cities, value=[cities[0]], size=8)
# Agrupar pontos por cidade
def pontos_por_cidade(selected_cities):
    teams_cidade = team[team['city'].isin(selected_cities)]
    df = pd.merge(teams_cidade[['id', 'city', 'latitude', 'longitude']], game, left_on='id', right_on='team_id_home', how='left')
    pontos = df.groupby(['city', 'latitude', 'longitude']).agg({'pts_home': 'sum'}).reset_index()
    fig = px.scatter_mapbox(pontos, lat='latitude', lon='longitude', size='pts_home', color='pts_home', hover_name='city', size_max=60, zoom=3, mapbox_style='open-street-map', title='Total de Pontos por Cidade (Home)')
    fig.update_layout(height=650, margin=dict(l=0, r=0, t=40, b=0))
    return pn.pane.Plotly(fig, config={'displayModeBar': True, 'scrollZoom': True}, height=650)

In [ ]:
# Filtro de data (usando game_info)
min_date = game_info['game_date'].min().date()
max_date = game_info['game_date'].max().date()
date_slider = pn.widgets.DateRangeSlider(name='Intervalo de Data dos Jogos', start=min_date, end=max_date, value=(min_date, max_date))
def pontos_por_data(date_range):
    # game_info não possui team_id_home, então precisamos usar game.csv para obter os times e pontos
    # Merge game_info com game.csv para obter os times de cada jogo
    df = pd.merge(game_info, game[['game_id','team_id_home','pts_home']], on='game_id', how='left')
    df = pd.merge(df, team[['id', 'city', 'latitude', 'longitude']], left_on='team_id_home', right_on='id', how='left')
    df = df[(df['game_date'] >= pd.to_datetime(date_range[0])) & (df['game_date'] <= pd.to_datetime(date_range[1]))]
    pontos = df.groupby(['city', 'latitude', 'longitude']).agg({'pts_home': 'sum'}).reset_index()
    fig = px.scatter_mapbox(pontos, lat='latitude', lon='longitude', size='pts_home', color='pts_home', hover_name='city', size_max=60, zoom=3, mapbox_style='open-street-map', title='Total de Pontos por Cidade (Filtrado por Data)')
    fig.update_layout(height=650, margin=dict(l=0, r=0, t=40, b=0))
    return pn.pane.Plotly(fig, config={'displayModeBar': True, 'scrollZoom': True}, height=650)

In [ ]:
# Layout final
geo_dashboard = pn.Column(
    pn.pane.Markdown('## Análise Geográfica'),
    pn.Row(
        pn.Column(
            pn.pane.Markdown('### Filtro por Cidade'),
            city_selector,
            pn.bind(pontos_por_cidade, city_selector),
        ),
        pn.Column(
            pn.pane.Markdown('### Filtro por Data dos Jogos'),
            date_slider,
            pn.bind(pontos_por_data, date_slider)
        ),
    ),
)


# Análise de Jogadores

In [1]:
import pandas as pd
import panel as pn
import plotly.express as px
pn.extension('plotly')
# Carregar dados de jogadores
players = pd.read_csv('csv/common_player_info.csv')
players.columns = [c.lower() for c in players.columns]
schools = sorted(players['school'].dropna().unique())
positions = sorted(players['position'].dropna().unique())

In [2]:
# Filtros dinâmicos
school_selector = pn.widgets.MultiSelect(name='Filtrar por Escola', options=schools, value=schools, size=8)
position_selector = pn.widgets.MultiSelect(name='Filtrar por Posição', options=positions, value=positions, size=8)

In [ ]:
def filtrar_jogadores(selected_schools, selected_positions):
    return players[(players['school'].isin(selected_schools)) & (players['position'].isin(selected_positions))]

In [ ]:
def grafico_school(schools, positions):
    df = filtrar_jogadores(schools, positions)
    contagem_school = df.groupby('school').agg({'person_id': 'count'}).reset_index()
    contagem_school = contagem_school.sort_values('person_id', ascending=False)
    fig = px.bar(contagem_school, x='person_id', y='school', orientation='h',
                 labels={'person_id': 'Contagem de person_id', 'school': 'school'},
                 title='Contagem de person_id por school',
                 height=400)
    fig.update_layout(margin=dict(l=0, r=0, t=40, b=0))
    return pn.pane.Plotly(fig, config={'displayModeBar': True}, height=400)

In [ ]:
def grafico_position(schools, positions):
    df = filtrar_jogadores(schools, positions)
    contagem_position = df.groupby('position').agg({'person_id': 'count'}).reset_index()
    contagem_position = contagem_position.sort_values('person_id', ascending=False)
    fig = px.bar(contagem_position, x='position', y='person_id',
                 labels={'person_id': 'Contagem de person_id', 'position': 'position'},
                 title='Contagem de Jogadores por posição',
                 height=300)
    fig.update_layout(margin=dict(l=0, r=0, t=40, b=0))
    return pn.pane.Plotly(fig, config={'displayModeBar': True}, height=300)

In [ ]:
# Layout organizado
player_analysis_dashboard = pn.Row(
    pn.Column(
        pn.pane.Markdown('## Filtros'),
        school_selector,
        position_selector,
        width=300,
    ),
    pn.Column(
        pn.bind(grafico_school, school_selector, position_selector),
        pn.bind(grafico_position, school_selector, position_selector),
        width=800,
    ),
)
player_analysis_dashboard

In [ ]:
pn.extension()
# Imagem de fundo (exemplo: URL pública, pode ser alterada para local se necessário)
background_url = 'https://m.media-amazon.com/images/S/sonata-images-prod/US_CHANNELS_NBA_LeaguePass_092225/22ef1594-50d9-459c-bf3d-7da4d8bcc873._UR1936,1089_SX2160_FMjpg_.jpeg'
html = f'''
<div style="position:relative;width:100%;height:600px;background:url('{background_url}') center/cover no-repeat;display:flex;flex-direction:column;justify-content:center;align-items:center;">
  <div style="background:rgba(0,0,0,0.6);padding:40px 60px;border-radius:16px;color:#fff;box-shadow:0 4px 24px #0006;">
    <h1 style="margin-bottom:16px;font-size:2.5em;">Dashboard NBA</h1>
    <h2 style="margin-bottom:24px;font-size:1.5em;">Trabalho: Análise de Dados da NBA</h2>
    <h3 style="margin-bottom:12px;">Integrantes do Grupo</h3>
    <ul style="font-size:1.2em;list-style:none;padding:0;">
      <li>Igor Gabriel Rodrigues</li>
      <li>Victor Mendonça Rodrigues</li>
      <li>Davi Farias de Freitas Manzotti</li>
      <li>Victor Hugo Queiroz Couto</li>
      <li>Rodrigo Debortoli de Souza</li>
    </ul>
  </div>
</div>
'''
home = pn.pane.HTML(html, width=1000, height=600)

In [ ]:
# Carregar dados
game = pd.read_csv('csv/game.csv', parse_dates=['game_date'])
team = pd.read_csv('csv/team.csv')
team.columns = [c.lower() for c in team.columns]
game.columns = [c.lower() for c in game.columns]
# Função para obter o nickname do time pelo id
def get_team_nickname(team_id):
    row = team[team['id'] == team_id]
    if not row.empty:
        return row.iloc[0]['nickname']
    return ''
# Criar coluna com nome do jogo
game['nome_jogo'] = game.apply(lambda row: f'{get_team_nickname(row["team_id_home"])} vs. {get_team_nickname(row["team_id_away"])} ({row["game_date"].strftime("%d/%m/%Y")})', axis=1)

In [ ]:
# Filtro de data e jogos
import datetime

data_min = game['game_date'].min()
data_max = game['game_date'].max()
start_default = max(pd.Timestamp('2020-01-01'), data_min)
end_default = data_max

date_range_slider = pn.widgets.DateRangeSlider(
    name='Intervalo de Data dos Jogos',
    start=data_min,
    end=data_max,
    value=(start_default, end_default),
    format='%d/%m/%Y',
    width=500
)

def jogos_filtrados():
    start, end = date_range_slider.value
    start = pd.Timestamp(start)
    end = pd.Timestamp(end)
    mask = (game['game_date'] >= start) & (game['game_date'] <= end)
    return game.loc[mask, 'nome_jogo'].tolist()

jogo_selector = pn.widgets.Select(
    name='Filtrar por Jogo',
    options=jogos_filtrados(),
    value=jogos_filtrados()[0] if jogos_filtrados() else None,
    width=400
)

def atualizar_jogos(event=None):
    opcoes = jogos_filtrados()
    jogo_selector.options = opcoes
    if opcoes:
        jogo_selector.value = opcoes[0]
    else:
        jogo_selector.value = None

date_range_slider.param.watch(atualizar_jogos, 'value')


In [ ]:
def get_jogo_info(nome_jogo):
    row = game[game['nome_jogo'] == nome_jogo].iloc[0]
    home_team = get_team_nickname(row['team_id_home'])
    away_team = get_team_nickname(row['team_id_away'])
    pts_home = row['pts_home']
    pts_away = row['pts_away']
    total_pontos = pts_home + pts_away
    return home_team, away_team, pts_home, pts_away, total_pontos

In [ ]:
def painel_jogo(nome_jogo):
    home_team, away_team, pts_home, pts_away, total_pontos = get_jogo_info(nome_jogo)
    style_card = """
        background:rgba(0,30,60,0.7);
        border-radius:32px;
        padding:32px 16px;
        color:#fff;
        box-shadow:0 4px 24px #0006;
        text-align:center;
        font-family:sans-serif;
        height:150px;
        display:flex;
        flex-direction:column;
        justify-content:center;
        align-items:center;
        width:250px;
    """
    style_title = 'font-size:2.5em;font-weight:bold;text-shadow:0 0 8px #00f;'
    style_label = 'font-size:1.2em;margin-bottom:8px;'
    style_points = 'font-size:2.5em;font-weight:bold;'
    style_total = 'font-size:2em;font-weight:bold;'
    pn.extension(raw_css=[
        """
        .jogo-row {
            background: linear-gradient(135deg, rgba(0,30,60,0.6), rgba(0,20,40,0.4));
            padding: 18px;
            border-radius: 20px;
            box-shadow: 0 8px 30px rgba(0,0,0,0.5);
            align-items: start;
        }
        .jogo-row .bk-root .bk-layout-cell {
            padding: 8px;
        }
        """
    ])

    return pn.Column(
        pn.Spacer(height=20),
        pn.Row(
            pn.Column(
                pn.pane.Markdown(f'<div style="{style_card}"><div style="{style_label}">Time da Casa</div><div style="{style_title}">{home_team}</div></div>'),
                pn.Spacer(height=20),
                pn.pane.Markdown(f'<div style="{style_card}"><div style="{style_label}">Pontos</div><div style="{style_points}">{pts_home:,.2f}</div></div>'),
            ),
            pn.Column(
                pn.Spacer(height=40),
                pn.pane.Markdown(f'<div style="{style_card}"><div style="{style_label}">Total de Pontos</div><div style="{style_total}">{total_pontos:,.0f}</div></div>'),
            ),
            pn.Column(
                pn.pane.Markdown(f'<div style="{style_card}"><div style="{style_label}">Time Visitante</div><div style="{style_title}">{away_team}</div></div>'),
                pn.Spacer(height=20),
                pn.pane.Markdown(f'<div style="{style_card}"><div style="{style_label}">Pontos</div><div style="{style_points}">{pts_away:,.2f}</div></div>'),
            ),
            css_classes=['jogo-row'],
        ),
        pn.Spacer(height=20),
        pn.Row(     
            pn.Spacer(width=100),
            pn.Column(
                pn.pane.Markdown('<div style="text-align:center;font-size:1.2em;color:#fff;">Intervalo de Data dos Jogos</div>'),
                date_range_slider,
                pn.pane.Markdown('<div style="text-align:center;font-size:1.2em;color:#fff;">Filtrar por Jogo</div>'),
                jogo_selector,
            ),
            pn.Spacer(width=100),
        ),
    )


In [ ]:
game_analysis_dashboard = pn.bind(painel_jogo, jogo_selector)

In [ ]:
import pandas as pd
import panel as pn
import plotly.express as px
pn.extension('plotly')
# Carregar dados
df_info = pd.read_csv('csv/common_player_info.csv')
df_player = pd.read_csv('csv/player.csv')
# Preprocessamento de altura e peso
def parse_height(h):
    if isinstance(h, str) and '-' in h:
        ft, inch = h.split('-')
        return round(int(ft) * 0.3048 + int(inch) * 0.0254, 2)
    return None
df_info['altura_m'] = df_info['height'].apply(parse_height)
df_info['peso_kg'] = pd.to_numeric(df_info['weight'], errors='coerce')
# Filtros
times = sorted(df_info['team_name'].dropna().unique().tolist())
posicoes = sorted(df_info['position'].dropna().unique().tolist())
time_selector = pn.widgets.Select(name='Time', options=['Todos'] + times, value='Todos', width=300)
pos_selector = pn.widgets.Select(name='Posição', options=['Todos'] + posicoes, value='Todos', width=300)
def filtrar():
    df = df_info.copy()
    if time_selector.value != 'Todos':
        df = df[df['team_name'] == time_selector.value]
    if pos_selector.value != 'Todos':
        df = df[df['position'] == pos_selector.value]
    return df
# Gráfico de distribuição de altura
def grafico_altura(time, pos):
    df = filtrar()
    fig = px.histogram(df, x='altura_m', nbins=20, labels={'altura_m': 'Altura (m)'}, title='Distribuição de Jogadores por Altura')
    fig.update_layout(height=350, plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)')
    fig.update_yaxes(title='Contagem de Jogadores')
    return fig
# Gráfico de distribuição de peso
def grafico_peso(time, pos):
    df = filtrar()
    fig = px.histogram(df, x='peso_kg', nbins=20, labels={'peso_kg': 'Peso (libras)'}, title='Distribuição de Jogadores por Peso')
    fig.update_layout(height=350, plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)')
    fig.update_yaxes(title='Contagem de Jogadores')
    return fig
# Gráfico de dispersão Altura vs Peso por posição
def grafico_dispersao(time, pos):
    df = filtrar()
    # Merge para pegar nome completo
    df = df.merge(df_player[['id', 'full_name']], left_on='person_id', right_on='id', how='left')
    fig = px.scatter(
        df,
        x='altura_m',
        y='peso_kg',
        color='position',
        hover_data={'full_name': True, 'position': True, 'altura_m': True, 'peso_kg': True},
        labels={'altura_m': 'Altura (m)', 'peso_kg': 'Peso (libras)', 'position': 'Posição'},
        title='Altura vs. Peso por Posição',
        height=400
    )
    fig.update_layout(plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)')
    return fig
# Painel final
physical_profile_dashboard = pn.Column(
    pn.Row(
        pn.Column(
            pn.pane.Markdown('<h2 style="color:#bdf;">Distribuição de Jogadores por Altura</h2>'),
            pn.bind(grafico_altura, time_selector, pos_selector),
        ),
        pn.Column(
            pn.pane.Markdown('<h2 style="color:#bdf;">Distribuição de Jogadores por Peso</h2>'),
            pn.bind(grafico_peso, time_selector, pos_selector),
        ),
        pn.Column(
            pn.pane.Markdown('<h2 style="color:#bdf;">Filtros</h2>'),
            pn.pane.Markdown('<b>Time</b>'),
            time_selector,
            pn.pane.Markdown('<b>Posição</b>'),
            pos_selector,
        ),
    ),
    pn.Row(
        pn.Column(
            pn.pane.Markdown('<h2 style="color:#bdf;">Altura vs. Peso por Posição</h2>'),
            pn.bind(grafico_dispersao, time_selector, pos_selector),
        ),
    ),
)

In [ ]:
pn.extension('tabulator')

# Dicionário de páginas
pages = {
    "Início": home,
    "Visão Geral": overview_dashboard,
    "Análise de Times": team_analysis_dashboard,
    "Análise Geográfica": geo_dashboard,
    "Análise de Jogadores": player_analysis_dashboard,
    "Análise de Jogos": game_analysis_dashboard,
    "Perfil Físico dos Jogadores": physical_profile_dashboard,
}

# Área principal dinâmica
main_area = pn.Column(pages["Início"])

# Função de callback
def show_page(event):
    main_area.objects = [pages[event.obj.name]]

# Cria botões verticais
menu_items = []
for name in pages:
    btn = pn.widgets.Button(name=name, button_type='primary', width=220)
    btn.on_click(show_page)
    menu_items.append(btn)

# Menu vertical (lista)
menu = pn.Column(*menu_items, sizing_mode="fixed", width=240)

# Template principal
dashboard = pn.template.FastListTemplate(
    site="Dashboard NBA",
    title="",
    sidebar=[
        pn.pane.Markdown("## Menu de Navegação"),
        menu
    ],
    main=[main_area],
    theme='dark'
)

dashboard.servable()
